# M1M3 Bump Test Overshoot
Craig Lage - 10-May-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
# Times of bump test.  According to Petr, we should use this test after the
# maxRateOfChange was reduced.
start = Time("2023-05-09T22:55:00", scale='utc')
end = Time("2023-05-10T02:00:00", scale='utc')

In [ ]:
# Bump test status
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
async def plotBumpTestFollowingErrors(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryFollow = f"primaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"primaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"secondaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryFollow = f"secondaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None
        secondaryForce = None
        secondaryFollow = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}", fontsize=18)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primaryForce, primaryFollow, 'timestamp'], start.utc, end.utc)
    timestamp = forces.index[0].isoformat().split('.')[0]
    plt.suptitle(f"Bump Test Following Errors. Actuator ID {id}\n {timestamp}", fontsize=18)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primaryForces = forces[primaryForce].values
    primaryErrors = forces[primaryFollow].values
    plotStart -= t0
    plotEnd -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, primaryForces, label='Data')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.title("Following Errors")
    plt.plot(times, primaryErrors)
    plt.text(5, 350, f"Max = {np.max(primaryErrors):.1f} nt")
    plt.text(5, -350, f"Min = {np.min(primaryErrors):.1f} nt")
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Following Errors (nt)")
    
    if secondaryName is not None:
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [secondaryForce, secondaryFollow, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondaryForces = forces[secondaryForce].values
        secondaryErrors = forces[secondaryFollow].values
        plotStart -= t0
        plotEnd -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondaryName}")
        plt.plot(times, secondaryForces, label='Data')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
        plt.legend()
        plt.subplot(2,2,4)
        plt.title("Following Errors")
        plt.plot(times, secondaryErrors)
        plt.text(5, 350, f"Max = {np.max(secondaryErrors):.1f} nt")
        plt.text(5, -350, f"Min = {np.min(secondaryErrors):.1f} nt")

        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400, 400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Following Errors (nt)")
    else:
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return

## First run just one actuator

In [ ]:
id = 112
fig = plt.figure(figsize=(10,10))
await plotBumpTestFollowingErrors(fig, bumps, id)
#plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Bump_Test_Residuals_{id}_28Apr23.png")

# Now run the whole bump test

In [ ]:
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/home/c/cslage/u/MTM1M3/data/Bump_Test_Following_Errors_{timestamp}.pdf")

for index in range(len(FATABLE)):
    try:
        id = FATABLE[index][M1M3FATable.FATABLE_ID]
        fig = plt.figure(figsize=(10,10))
        await plotBumpTestFollowingErrors(fig, bumps, id)
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.close()
    except:
        print(f"{id} failed!")
        continue
pdf.close()


In [ ]:
id = 112
thisBump = bumps[bumps['actuatorId']==id]
index = M1M3FATable.actuatorIDToIndex(id)
primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
primaryFollow = f"primaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
primaryMeasuredForce = f"primaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
primaryAppliedForce = f"primaryCylinderForces{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
    secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
    secondaryForce = f"secondaryCylinderForces{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
    #secondaryFollow = f"secondaryCylinderFollowingError{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
    secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
else:
    secondaryName = None
    secondaryForce = None
    secondaryFollow = None
plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
plotEnd = plotStart + 13.0 
start = Time(plotStart, format='unix_tai', scale='tai')
end = Time(plotEnd, format='unix_tai', scale='tai')
appliedForces = await client.select_time_series("lsst.sal.MTM1M3.appliedCylinderForces", \
                                         [primaryAppliedForce, 'timestamp'], start.utc, end.utc)
measuredForces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [primaryMeasuredForce, primaryFollow, 'timestamp'], start.utc, end.utc)
timestamp = forces.index[0].isoformat().split('.')[0]
appliedTimes = appliedForces['timestamp'].values
at0 = appliedTimes[0]
appliedTimes -= at0
measuredTimes = measuredForces['timestamp'].values
mt0 = measuredTimes[0]
measuredTimes -= at0
 

In [ ]:
len(measuredForces)

In [ ]:
plt.plot(appliedTimes, appliedForces[primaryAppliedForce] / 1000.0)
plt.plot(measuredTimes, measuredForces[primaryMeasuredForce])
plt.plot(measuredTimes, measuredForces[primaryFollow])